## Algoritmo Apriori

### Carga das bibliotecas

In [26]:
# Bibliotecas
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

### Base de Dados

In [58]:
# --- 1. Leitura dos dados ---

df = pd.read_csv('vendas_calcados.csv')


### Visualização do Dataframe

In [59]:
df.columns

Index(['Unnamed: 0', 'vendedor', 'produto', 'cliente', 'cidade', 'mês'], dtype='object')

In [60]:
df.head()

,Unnamed: 0,vendedor,produto,cliente,cidade,mês
0,0,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM FEMININO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro
1,1,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM FEMININO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro
2,2,V.V. REPRESENTACAO COM DE CALCADOS LTDA,RECEM NASCIDO ALGO MAIS,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Novembro
3,3,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM MASCULINO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro
4,4,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM MASCULINO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro


In [61]:
df.isnull().sum()

Unnamed: 0    0
vendedor      0
produto       0
cliente       0
cidade        0
mês           0
dtype: int64

In [62]:
# --- 2. Mapeamento de meses para trimestres ---
mes_para_trimestre = {
    'Janeiro': 'T1', 'Fevereiro': 'T1', 'Março': 'T1',
    'Abril': 'T2', 'Maio': 'T2', 'Junho': 'T2',
    'Julho': 'T3', 'Agosto': 'T3', 'Setembro': 'T3',
    'Outubro': 'T4', 'Novembro': 'T4', 'Dezembro': 'T4'
}

df['trimestre'] = df['mês'].map(mes_para_trimestre)

In [78]:
df.head()

,Unnamed: 0,vendedor,produto,cliente,cidade,mês,trimestre
0,0,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM FEMININO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro,T1
1,1,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM FEMININO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro,T1
2,2,V.V. REPRESENTACAO COM DE CALCADOS LTDA,RECEM NASCIDO ALGO MAIS,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Novembro,T4
3,3,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM MASCULINO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro,T1
4,4,V.V. REPRESENTACAO COM DE CALCADOS LTDA,MOCASSIM MASCULINO,A. A. MUSTAFA COM DE MODAS LTDA EPP,SAO PAULO - SP,Janeiro,T1


### Escolha das colunas para o Apriori

In [72]:
# --- 3. Agrupamento por cliente + trimestre ---
grouped = df.groupby(['cliente', 'trimestre'])['produto'].apply(list).reset_index()

# Guarda o conjunto de dados em Html
grouped.to_html('dados.html')

# Guarda o conjunto de dados em csv
grouped.to_csv('dados.csv')

In [79]:
grouped.head()

,cliente,trimestre,produto
0,2M CALCADOS E CONFECCOES LTDA,T2,"[CALCADO KLIN TENIS FEMININO ALGODAO DOCE 115,..."
1,3M - LOJAS FREE - EIRELI,T3,[CALCADO KLIN MOCASSIM MASCULINO RECEM NASCIDO...
2,A A DOS SANTOS & CIA LTDA ME,T2,"[CALCADO KLIN MOCASSIM MASCULINO CURUMIM 054, ..."
3,A ALMEIDA COSTA SUBRINHO,T1,"[MOCASSIM MASCULINO, MOCASSIM MASCULINO, SANDA..."
4,A ALMEIDA COSTA SUBRINHO,T3,[CALCADO KLIN SAPATILHA FEMININA RECEM NASCIDO...


### Execução do Apriori

In [73]:
# --- 4. Criar lista de transações (lista de listas) ---
list_of_lists = grouped['produto'].tolist()

# --- 5. Transformação binária com TransactionEncoder ---
te = TransactionEncoder()
te_ary = te.fit(list_of_lists).transform(list_of_lists)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# --- 6. Execução do Apriori ---
support = 0.01
confidence = 0.85

frequent_item_sets = apriori(df_encoded, min_support=support, use_colnames=True)
rules = association_rules(frequent_item_sets, metric="confidence", min_threshold=confidence)

# Suporte do conjunto de itens frequentes 
support = frequent_item_sets.sort_values(by = ['support'], ascending = False)

# Regras de associação geradas
results = rules.sort_values(
    by = ['lift'], ascending = False).drop(['zhangs_metric', 'antecedent support', 'consequent support', 'leverage', 
                                            'conviction'],  axis = 1) 

# Gravação dos resultados para visualização do usuário

# Salva o Suporte do conjunto de itens frequentes em HTML para análise de ocorrências
support.to_html('support.html', index = False)

# Salva as regras de associação em HTML para análise do resultado
results.to_html('association_rules.html', index = False)

In [ ]:
# Filtrar regras com poucos itens no antecedente e com suporte acima de 2%
results_few_items = results[results['antecedents'].apply(lambda x: len(x) <= 2)]

results_few_items = results_few_items[results_few_items['support'].apply(lambda x: x >= 0.02)]

# Ordenar para facilitar análise
results_few_items = results_few_items.sort_values(by='lift', ascending=False)

# Exportar para visualização
results_few_items.to_html('association_rules_few_antecedents.html', index=False)


### Visualização dos resultados

In [80]:
# Lista de lista: conjunto de dados em formato de matriz
list_of_lists[0:10]

[['CALCADO KLIN TENIS FEMININO ALGODAO DOCE 115',
  'CALCADO KLIN TENIS FEMININO ALGODAO DOCE 115',
  'CALCADO KLIN SAPATILHA FEMININA ALGODAO DOCE 119'],
 ['CALCADO KLIN MOCASSIM MASCULINO RECEM NASCIDO B001',
  'CALCADO KLIN SANDALIA MASCULINA RECEM NASCIDO B016',
  'CALCADO KLIN SAPATILHA FEMININA RECEM NASCIDO B055',
  'CALCADO KLIN SANDALIA FEMININA RECEM NASCIDO B073',
  'CALCADO KLIN MOCASSIM MASCULINO CURUMIM 059',
  'CALCADO KLIN MOCASSIM MASCULINO CURUMIM 063',
  'CALCADO KLIN SANDALIA MASCULINA RECEM NASCIDO B012',
  'CALCADO KLIN SANDALIA MASCULINA RECEM NASCIDO B014',
  'CALCADO KLIN SAPATO FEMININA RECEM NASCIDO B040',
  'CALCADO KLIN SANDALIA FEMININA RECEM NASCIDO B045',
  'CALCADO KLIN SAPATILHA FEMININA RECEM NASCIDO B072',
  'CALCADO KLIN SAPATILHA FEMININA RECEM NASCIDO B072',
  'CALCADO KLIN MOCASSIM MASCULINO RECEM NASCIDO B009',
  'CALCADO KLIN SAPATO MASCULINO RECEM NASCIDO B003',
  'CALCADO KLIN SANDALIA FEMININA RECEM NASCIDO B036',
  'CALCADO KLIN SANDALIA MA

In [55]:
# Data Frame: conjunto de dados em formato binário
df_encoded

,ALGODAO DOCE,ALPARGATAS RECEM NASCIDO FIT,BOTA MASCULINA,BOTA CASUAL MASCULINA,BOTA FEMININA,BOTA FEMININO,BOTA MASCULINA,CALCADO KLIN SAPATO MASCULINO RECEM NASCIDO 188,CALCADO DELICAT MOCASSIM FEMININO 020,CALCADO DELICAT MOCASSIM FEMININO 021,...,TENIS MASCULINO PRIMEIROS PASSOS 013,TENIS RECEM NASCIDO FIT,TENIS SAMEKA INFANTIL FEMININO 001,TENIS SAMEKA INFANTIL FEMININO 002,TENIS SAMEKA INFANTIL FEMININO 003,TENIS SAMEKA INFANTIL MASCULINO 001,TENIS SAMEKA INFANTIL MASCULINO 003,TENIS UNISSEX PRIMEIROS PASSOS 001,TENIS UNISSEX PRIMEIROS PASSOS 002,TENIS UNISSEX PRIMEIROS PASSOS 003
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8542,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8543,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [75]:
# Suporte do conjunto de itens frequentes
support.count()

support     498
itemsets    498
dtype: int64

In [76]:
# Resultados com suporte 0.01, confiança de 0.85 sem filtro de antecedentes
results.count()

antecedents         37
consequents         37
support             37
confidence          37
lift                37
representativity    37
jaccard             37
certainty           37
kulczynski          37
dtype: int64

In [77]:
# Resultados com suporte filtrado de 0.02, confiança de 0.85 com filtro de antecedentes <= 2
results_few_items.count()

antecedents         5
consequents         5
support             5
confidence          5
lift                5
representativity    5
jaccard             5
certainty           5
kulczynski          5
dtype: int64